In [1]:
pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installatio

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, accuracy_score
import seaborn as sns

In [3]:
IMAGE_SIZE_1=224
IMAGE_SIZE_2=224
BATCH_SIZE=32
EPOCHS=50
CHANNELS = 3
dataset_path = "/content/drive/MyDrive/PharmaScope"

In [4]:
# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    shuffle=True,
    image_size=(IMAGE_SIZE_1, IMAGE_SIZE_2),
    batch_size=BATCH_SIZE
)

Found 3495 files belonging to 27 classes.


In [5]:
class_names = dataset.class_names
class_names

['Albendazole',
 'Amoxicillin',
 'Beclovent dry powder capsule',
 'Bisacodyl',
 'Cephalexin',
 'Cetirizine',
 'Cypro',
 'Dexamethasone',
 'Diclofenac',
 'Domperidone',
 'Famotidine',
 'Flutihale dry powder capsule',
 'Folic acid',
 'Iprahale dry powder capsule',
 'Losatan',
 'Metformin',
 'Nifitepin',
 'Not Medicine',
 'Omeprazole',
 'Paracetamol',
 'Piriton',
 'Propranolol',
 'Salbutamol',
 'Salhale dry powder capsule',
 'Vitamin C',
 'Vitamin b-complex',
 'gastro resistant tablets']

In [6]:
len(dataset)

110

In [7]:
def get_dataset_partions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):

    ds_size=len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size= int(train_split * ds_size)
    val_size= int(val_split * ds_size)

    train_ds= ds.take(train_size)

    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [8]:
train_ds, val_ds, test_ds = get_dataset_partions_tf(dataset)

In [9]:
len(test_ds)

11

In [10]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

In [11]:
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [12]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
vgg16 = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(IMAGE_SIZE_1, IMAGE_SIZE_2, CHANNELS)
)

58889256/58889256 [==============================] - 0s 0us/step


In [14]:
# Freeze the VGG16 layers
for layer in vgg16.layers:
    layer.trainable = False

In [15]:
model = models.Sequential([
    vgg16,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])


In [16]:
model.build(input_shape=(None, IMAGE_SIZE_1, IMAGE_SIZE_2, CHANNELS))

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 27)               

In [21]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0007),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [22]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [24]:
# Train the model
history = model.fit(train_ds, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_data=val_ds, callbacks=[early_stopping])

Epoch 1/50
26/88 [=======>......................] - ETA: 5s - loss: 1.6312 - accuracy: 0.6106

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
scores = model.evaluate(test_ds)

In [ ]:
# Extract training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
# Plot training & validation accuracy and loss
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(len(acc)), acc, label='Training Accuracy')
plt.plot(range(len(val_acc)), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(len(loss)), loss, label='Training Loss')
plt.plot(range(len(val_loss)), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
# Function to predict and plot images with actual and predicted labels
def predict_and_plot(model, dataset):
    plt.figure(figsize=(15, 15))
    for images, labels in dataset.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            img_array = tf.expand_dims(images[i].numpy(), 0)
            predictions = model.predict(img_array)
            predicted_class = class_names[np.argmax(predictions[0])]
            actual_class = class_names[labels[i]]
            confidence = round(100 * np.max(predictions[0]), 2)
            plt.title(f"Actual: {actual_class}, \nPredicted: {predicted_class}, \nConfidence: {confidence:.2f}%")
            plt.axis("off")

In [ ]:
def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')
# Smooth the training and validation accuracy and loss
window_size = 11  # Adjust the window size for more or less smoothing
smooth_acc = moving_average(acc, window_size)
smooth_val_acc = moving_average(val_acc, window_size)
smooth_loss = moving_average(loss, window_size)
smooth_val_loss = moving_average(val_loss, window_size)

# Adjust the range to match the smoothed data
smooth_range = range(len(smooth_acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(smooth_range, smooth_acc, label='Training Accuracy')
plt.plot(smooth_range, smooth_val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('raining and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(smooth_range, smooth_loss, label='Training Loss')
plt.plot(smooth_range, smooth_val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
# Function to calculate F1 score, precision, and recall
def calculate_metrics(model, dataset):
    y_true = []
    y_pred = []

    for images, labels in dataset:
        y_true.extend(labels.numpy())
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))
    return y_true, y_pred



In [ ]:
# Calculate precision, recall, F1 score
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, accuracy_score
y_true, y_pred = calculate_metrics(model, test_ds)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

In [ ]:
# Print metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("Accuracy:", accuracy)


In [ ]:
# Function to calculate F1 score and plot confusion matrix
def calculate_metrics(model, dataset):
    y_true = []
    y_pred = []

    for images, labels in dataset:
        y_true.extend(labels.numpy())
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix")
print(cm)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix with colors
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)  # Adjust font scale for better visualization

# Create a heatmap with annotations and color bar
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=True,
            xticklabels=class_names, yticklabels=class_names)


plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Print classification report
cr = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report")
print(cr)


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

def calculate_metrics(y_true, y_pred):
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Calculate classification report
    cr = classification_report(y_true, y_pred)

    # Calculate precision, recall, f1-score (weighted average)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    return cm, cr, precision, recall, f1_score, accuracy

In [ ]:
# Calculate metrics
cm, cr, precision, recall, f1_score, accuracy = calculate_metrics(y_true, y_pred)

# Print metrics
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(cr)
print("\nPrecision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
print("Accuracy:", accuracy)

In [ ]:
# Save the model
model.save(f"/content/drive/MyDrive/CorrectModels/VGG16v1.h5")